# Introdução
Esse arquivo foi feito para usar as analises em conjunto de forma estruturada para encontrar as melhores combinações de números baseado na frequencia e no intervalo de desaparecimento dos mesmos.

Para isso foi utilizado as sequintes analises:
* Frequência total de cada número
* Todos os intervalos de desaparecimento de cada número

Ainda estou aperfeiçoando a analise com o objetivo de melhorar a precisão das combinações posseiveis, estou estudando mais estatisca, machine learning e analise de dados para obter exito.

Pretendo criar outras analises para complementar e estrutura ainda mais as analises para ser mais robusto a tomada de decisão.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import bar_chart_race as bcr
import matplotlib as mpl
import os
from openpyxl import load_workbook
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4

caminho = r"c:\Users\castr\OneDrive\Área de Trabalho\Repositorios\scripts\Analise-loterias\data\processed"

os.chdir(caminho)

mega = pd.read_excel('Dados_Update.xlsx')

table_for_analysis = mega.drop(['Concurso', 'Data'], axis=1)


## Frequência total e individual de cada bola

Código abaixo gera uma tabela com as frequencias de cada número por bola e frequencia total 

In [ ]:
contagem = {}

for numero in range(1, 61):
    contagem_numero = {}
    for coluna in table_for_analysis.columns:
        temp = 0
        for instancia in table_for_analysis[coluna]:
            if instancia == numero:
                temp += 1
        
        contagem_numero[coluna] = temp
    
    contagem[numero] = contagem_numero

tabela = pd.DataFrame(contagem).T

tabela.index.name = 'Número'

tabela['Total'] = tabela.sum(axis=1)

caminho_original = os.getcwd()

os.chdir(r"c:\Users\castr\OneDrive\Área de Trabalho\Repositorios\scripts\Analise-loterias\reports\Metricas\MegaSena")

tabela.to_excel("tabela_frequencias.xlsx", sheet_name="Frequencia")

os.chdir(caminho_original)

display(tabela)

## Tempo de desaparecimento

O código abaixo gera todos os intervalos de cada número onde cada um ficou desaparecido

In [ ]:
numeros = {i: [] for i in range(1, 61)}

contadores = {i: 0 for i in range(1, 61)}

for num in range(1, 61):
    for index, row in table_for_analysis.iterrows():

        if num in row.values:

            numeros[num].append(contadores[num])
            
            contadores[num] = 0
        else:
            
            contadores[num] += 1

tabela_Total_Desaparecimento = pd.DataFrame.from_dict(numeros, orient='index')

tabela_Total_Desaparecimento.to_excel('Intervalos de Desaparecimento.xlsx', sheet_name='Intervalos Históricos')

display(tabela_Total_Desaparecimento)


'c:\\Users\\castr\\OneDrive\\Área de Trabalho\\Repositorios\\scripts\\Analise-loterias\\data\\processed'

## Complemento do tempo de desaparecimento

O código abaixo retorna o maior e menor intervalo de tempo que um número ficou desaparecido, como bónus ele também dá a media de desaparecimento de cada número 

In [29]:
lista = {}
maximo = {}

for i in range(1,61):
    valores = numeros.get(i)
    
    lista[i] = round(sum(valores) / len(valores), 2)

    maximo[i] = max(valores)

tabela_Media_Desaparecimento = pd.DataFrame.from_dict(lista, orient='index', columns=['Tempo de Desaparecimento Médio'])

tabela_Desaparecimento_maximo = pd.DataFrame.from_dict(maximo, orient='index', columns=['Tempo de Desaparecimento Máximo'])

os.chdir(r"c:\Users\castr\OneDrive\Área de Trabalho\Repositorios\scripts\Analise-loterias\reports\Metricas\MegaSena")

with pd.ExcelWriter("Intervalos de Desaparecimento.xlsx", engine='openpyxl',mode='a', if_sheet_exists='replace') as writer:
    tabela_Media_Desaparecimento.to_excel(writer, sheet_name='Média Desaparecimento', index=True, )
    tabela_Desaparecimento_maximo.to_excel(writer, sheet_name='Máximo Desaparecimento', index=True)